# Influence Maximization at Scale (with Fairness)

## Resources
This notebook is based off a few resources:

[Data](https://drive.google.com/file/d/1AFuShgAdyoqodqR1oFlCRp7okEYDdeLt/view) - 20 GB

**Papers:**
1. Primary: https://arxiv.org/pdf/2306.01587.pdf
2. Secondary: https://arxiv.org/abs/1904.08804

**Code:**
1. Primary: https://github.com/yu-ting-feng/fair_at_scale
2. Secondary: https://github.com/geopanag/IMINFECTOR/tree/master


## Pre-Requisites:
1. This notebook must run in a Google Collab environment with [Pro subscription](https://colab.research.google.com/signup) .
2. The data must be stored in a google drive account on which the user has full permissions to read / write. This notebook will attempt to connect to user's google drive and write to it.
3. The storage requirements within Google drive must be > 50 GB of write-able space.
4. The memory requirements of this notebook are high, the pro subscription currently gives access to 50 GB worth of memory, which is a good starting point.

## Google Drive Setup

Create a directory called `FairInfluenceMaximization` under which there are 2 sub-directories:
1. data
2. code

The data directory should contain the Data.zip (20 GB data file)

In [17]:
# Ensure you have enough RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [18]:
!pip install google-colab
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [34]:
# DO NOT CHANGE THIS
parent_path = "/gdrive/MyDrive/FairInfluenceMaximization/"
data_path = "/gdrive/MyDrive/FairInfluenceMaximization/data/"
code_path = "/gdrive/MyDrive/FairInfluenceMaximization/code/"
embeddings_folder = "/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Embeddings/"

In [35]:
!mkdir -p {data_path}
!mkdir -p {code_path}
!mkdir -p {embeddings_folder}

### Data Copy and Unzip

Now copy the Data.zip file into `/gdrive/MyDrive/FairInfluenceMaximization/data/`

In [21]:
contents = !ls {data_path}
contents = str(contents)
contents

"['Data  Data.zip\\tweibodata.zip  weibo_network.txt']"

In [22]:
# if Data.zip has been unzipped, then it would occur twice
count_data = contents.count("Data")

In [23]:
import os
data_zip = "Data.zip"

# At least Data.zip should be present
if data_zip not in contents:
  raise Exception(f"Please copy over {data_zip} into the folder: {data_path}")

if os.path.exists(data_path) and count_data == 1 and data_zip in contents:
  !unzip Data.zip

In [8]:
# assert that you have unzipped the folder
contents = !ls {data_path}
contents = str(contents)
count_data = contents.count("Data")
assert(count_data > 1)

In [12]:
%cd {data_path}
weibo_network_rar_path = data_path + "Data/Weibo/Init_Data/weibo_network.rar"
!unrar x {weibo_network_rar_path} -Y

/gdrive/MyDrive/FairInfluenceMaximization/data

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/weibo_network.rar

Extracting  weibo_network.txt                                              0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 80% 81% 82% 83% 84% 85% 86% 

In [13]:
# copy locally all large files as reading from disk over mount is very slow
!mkdir /opt/data/
!rm -rf /opt/data/*
!ls /opt/data/*

weibo_network_txt_path = data_path + "Data/Weibo/Init_Data/weibo_network.txt"
!rsync -avz --progress {weibo_network_txt_path} /opt/data/

# ensure the copy was successful
!diff -q {weibo_network_txt_path} /opt/data/

mkdir: cannot create directory ‘/opt/data/’: File exists
ls: cannot access '/opt/data/*': No such file or directory
sending incremental file list
weibo_network.txt
  3,739,600,920 100%   96.97MB/s    0:00:36 (xfr#1, to-chk=0/1)

sent 925,705,096 bytes  received 35 bytes  24,685,470.16 bytes/sec
total size is 3,739,600,920  speedup is 4.04


In [14]:
### ONLY RUN ONCE - setup
# abhisha: probably don't need
%cd /gdrive/MyDrive/FairInfluenceMaximization/data/weibodata/
!unrar x /gdrive/MyDrive/FairInfluenceMaximization/data/weibodata/

[Errno 2] No such file or directory: '/gdrive/MyDrive/FairInfluenceMaximization/data/weibodata/'
/gdrive/MyDrive/FairInfluenceMaximization/data

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal

Cannot open /gdrive/MyDrive/FairInfluenceMaximization/data/weibodata/.rar
No such file or directory
No files to extract


### Repo Setup

In [10]:
# clone repos
%cd {code_path}
!git clone https://github.com/geopanag/IMINFECTOR.git
!git clone https://github.com/abhisha1991/fair_at_scale.git

/gdrive/MyDrive/FairInfluenceMaximization/code
fatal: destination path 'IMINFECTOR' already exists and is not an empty directory.
Cloning into 'fair_at_scale'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 32 (delta 9), reused 30 (delta 7), pack-reused 0
Receiving objects: 100% (32/32), 16.98 KiB | 2.12 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [15]:
repo_im_fairness_path = code_path + "fair_at_scale/"
repo_iminfector_path = code_path + "IMINFECTOR/"

In [16]:
# install libraries
iminfector_requirements_path = code_path + "IMINFECTOR/requirements.txt"
%cat {iminfector_requirements_path}
%pip install networkx
%pip install pandas
%pip install numpy
%pip install tensorflow
%pip install igraph
%pip install pyunpack
%pip install patool

networkx==2.3
pandas==0.24.2
numpy==1.16.4
tensorflow==1.15.2
igraph==0.7.1
pyunpack==0.1.2
patool==1.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 4.0 MB/s eta 0:00:00


In [24]:
# import infector libraries
repo_im_fairness_models_path = repo_im_fairness_path + "models/"
%cd {repo_im_fairness_models_path}

import argparse
import os
import time
import sys

%set_env PYTHONPATH=/env/python:{repo_iminfector_path}
sys.path.append(repo_iminfector_path)

# resolved from the IMINFECTOR repository after adding env path
import evaluation
import extract_feats_and_trainset
import iminfector
import infector
import preprocess_for_imm
import preprocessing
import rank_nodes
import fair_iminfector

# imports
import json
import os
import time
from datetime import datetime
from typing import Dict
import igraph as ig
import numpy as np
from collections import defaultdict
import math
# import pandas after everything else
import pandas as pd

/gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models
env: PYTHONPATH=/env/python:/gdrive/MyDrive/FairInfluenceMaximization/code/IMINFECTOR/


In [26]:
# confirm file locations
!find /gdrive/MyDrive/FairInfluenceMaximization/data/ -type f -name 'train_cascades.txt'
!find /gdrive/MyDrive/FairInfluenceMaximization/data/ -type f -name 'test_cascades.txt'
!find /gdrive/MyDrive/FairInfluenceMaximization/data/ -type f -name 'active_users.txt'
!find /gdrive/MyDrive/FairInfluenceMaximization/data/ -type f -name 'weibo_network.txt'
!find /gdrive/MyDrive/FairInfluenceMaximization/data/ -type f -name 'graph_170w_1month.txt'
!find /gdrive/MyDrive/FairInfluenceMaximization/data/ -type f -name 'uidlist.txt'

/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/train_cascades.txt
/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/test_cascades.txt
/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/active_users.txt
/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/weibo_network.txt
/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/graph_170w_1month.txt
/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/uidlist.txt
/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/diffusion/uidlist.txt


In [27]:
# set parameters, check that critical files can be found
def get_parameters():
    """
    This function creates and gets arguments for sampling percentage, learning rate, number of epochs,
    embedding size, and number of negative samples.

    :return: list with argument values for sampling percentage, learning rate, number of epochs, embeddings size, and number of negative samples
    """
    sampling_perc = 120
    learning_rate = 0.1
    n_epochs = 10
    embedding_size = 50
    num_neg_samples =10

    return int(sampling_perc), float(learning_rate), int(n_epochs), int(embedding_size), int(
        num_neg_samples)


sampling_perc, learning_rate, n_epochs, embedding_size, num_neg_samples = get_parameters()
print('params=', sampling_perc, learning_rate, n_epochs, embedding_size, num_neg_samples)

print('size of train_cascades.txt ', os.path.getsize(f"{data_path}Data/Weibo/Init_Data/train_cascades.txt"))
print('FAC folder exists ', os.path.isdir(f"{data_path}Data/Weibo/Init_Data/FAC"))

if (not os.path.isfile(f"{data_path}Data/Weibo/Init_Data/train_cascades.txt")) \
  or (not os.path.isdir(f"{data_path}Data/Weibo/Init_Data/FAC")):
    print('Required files missing - RUN preprocessing')
else:
  print('Required files available - SKIP preprocessing')

#preprocessing.run(input_fn, input_log)
  #or (
#        not os.path.isdir(os.getcwd() + "/Weibo/Init_Data/FAC")):
#    preprocessing.run(input_fn, input_log)
#extract_feats_and_trainset.run(input_fn, 'region', sampling_perc, input_log)
# preprocess_for_imm.run(input_fn, input_log)
# rank_nodes.run(input_fn)
# infector.run(input_fn, learning_rate, n_epochs, embedding_size, num_neg_samples, input_log)
# iminfector.run(input_fn, embedding_size, input_log)
#evaluation.run2(input_fn, input_log, 'region')


params= 120 0.1 10 50 10
size of train_cascades.txt  390587429
FAC folder exists  False
Required files missing - RUN preprocessing


In [28]:
# define fairness calculation functions
def remove_duplicates(cascade_nodes, cascade_times):
    """
    Some tweets have more then one retweets from the same person
    Keep only the first retweet of that person
    """
    duplicates = set([x for x in cascade_nodes if cascade_nodes.count(x) > 1])
    for d in duplicates:
        to_remove = [v for v, b in enumerate(cascade_nodes) if b == d][1:]
        cascade_nodes = [b for v, b in enumerate(cascade_nodes) if v not in to_remove]
        cascade_times = [b for v, b in enumerate(cascade_times) if v not in to_remove]

    return cascade_nodes, cascade_times

def mapped_uid():
    '''
    map user id from uidlist.txt
    :return: dict of user id map
    '''
    #file_path = '/gdrive/MyDrive/FairInfluenceMaximization/data/weibodata/uidlist.txt'
    file_path = f'{data_path}Data/Weibo/Init_Data/uidlist.txt'
    with open(file_path, "r", encoding="gbk") as f:
        lines_uid = f.readlines()
    uid_map = {}
    for idx, uid in enumerate(lines_uid):
        uid_map[uid.strip()] = idx

    return uid_map



def get_attribute_dict(fn:str, path: str, attribute: str) -> Dict:
    """
    This function creates a gender dictionary using the profile_gender.csv if the file is available. If the file
    isn't available, it calls the generate_profile_gender_csv() function to generate the CSV and then builds the
    dictionary.

    :param path: path to profile_gender.csv
    :return: gender_dict: dictionary with user IDs as keys and 0 or 1 values indicating that the user is female or male
    """

    try:
        with open(path, 'r', encoding="ISO-8859-1") as f:
            contents = f.read()
    except:

        print('ERROR cannot find: ', path)
        return

        # path_user_profile = '/'.join(path.split("/")[:-1]) + "/userProfile/"
        #path_user_profile = '/gdrive/MyDrive/FairInfluenceMaximization/data/weibodata/userProfile/' ####
        path_user_profile = f'{data_path}Data/Weibo/Init_Data/userProfile/' ####

        txt_files = [os.path.join(path_user_profile, f) for f in os.listdir(path_user_profile) if
                     os.path.isfile(os.path.join(path_user_profile, f))]
        user_profile_df = pd.DataFrame()
        for t in txt_files:
            with open(t, 'r', encoding="ISO-8859-1") as f:
                contents = f.read()
            split_content = contents.split('\n')[:-1]

            reshaped_content = np.reshape(split_content, (int(len(split_content) / 15), 15))
            df = pd.DataFrame(reshaped_content)
            df = df.rename(columns=df.iloc[0]).drop(df.index[0])
            df.columns = df.columns.str.lstrip("# ")
            user_profile_df = user_profile_df.append(df)

        attribute_df = user_profile_df[["id", attribute]].reset_index(drop=True)
        uid_map = mapped_uid()
        attribute_df.id = attribute_df.id.map(uid_map)  # mapping user id

        if attribute == 'gender' and fn == 'weibo':
            gender_conversion_dict = {"m": 1, "f": 0}
            attribute_df[attribute] = attribute_df[attribute].map(gender_conversion_dict)

        attribute_df.to_csv(path, index=False)  # store the processed data

        attribute_dict = pd.Series(attribute_df[attribute].values, index=attribute_df.id).to_dict()

        return attribute_dict

    split_content, split_content_list = contents.split('\n')[0:-1], []
    for i in split_content:
        split_content_list.append(i.split(","))
    split_content_list = split_content_list[1:]

    attribute_dict = {}
    for split_data in split_content_list:
        attribute_dict[split_data[0]] = int(split_data[1])

    return attribute_dict

def compute_coef(L):
    #print('@compute_coef - why does mean calc take so long?')
    #print('size of L: ', len(L))
    #print('mean of L: ', np.mean(L))
    #print('stdev of L:', np.std(L, dtype=np.float64))
    sigma = np.std(L, dtype=np.float64)
    #sigma = np.sqrt(np.mean([(L[i]-np.mean(L))**2 for i in range(len(L))])) # strandard deviation
    #print('done 1')
    # abhisha: this is the coeff of variation from the paper, L is the influence ratio
    coef = sigma/np.mean(L) # coef of variation
    #print('done 2')
    sigmoid = 1 / (1 + math.e ** -coef)
    #print('done 3')
    return  2*(1-sigmoid)# sigmod function


def compute_fair(node_list, attribute_dict, grouped, attribute='gender'):
    '''
    :param node_list: cascade nodes
    :param attribute_dict: original attribute dict
    :param grouped: statistics of attribute dict
    :return: fairness score
    '''

    # influenced statistics
    influenced_attribute_dict = {k: attribute_dict[k] for k in node_list if k in user_attribute_dict}
    # abhisha: T_grouped is the male population influenced
    # abhisha: grouped is the total male population
    T_grouped = defaultdict(list)
    for k, v in influenced_attribute_dict.items():
        T_grouped[v].append(k)

    ratio = [len(T_grouped[k]) / len(grouped[k]) for k in grouped.keys()]

    #print('about to compute_coef')
    score = compute_coef(ratio)
    #print('finished compute_coef score: ', score)
    # abhisha: province here is region
    if attribute == 'province':
        min_f = 0.00537
        k = 0.566 # coefficient of scaling get from distribution [0.5,1] a=0.5, b=1, k = (b-a)/(max(score)-min(score))
        score = 0.5 + k * (score-min_f) # 0.5 min scaling border

    return score

def store_samples(fn, cascade_nodes, cascade_times, initiators, train_set, op_time, attribute_dict, grouped, attribute, sampling_perc=120):
    """
    Store the samples  for the train set as described in the node-context pair creation process for INFECTOR
    """
    # ---- Inverse sampling based on copying time
    # abhisha: this is for FPS - we're oversampliing first
    no_samples = round(len(cascade_nodes) * sampling_perc / 100)
    casc_len = len(cascade_nodes)
    # abhisha: this is the time penalty
    times = [1.0 / (abs((cascade_times[i] - op_time)) + 1) for i in range(0, casc_len)]
    s_times = sum(times)

    #print('@store_samples ', 'no_samples=', no_samples, 'casc_len=', casc_len)

    f_score = compute_fair(cascade_nodes,attribute_dict,grouped, attribute)

    #print('@store_samples ', 'f_score=', f_score)
    if (f_score is not None) and (not np.isnan(f_score)):
        if s_times == 0:
            samples = []
        else:
            #print("out")
            # abhisha: this is the p(v|d) from the paper
            probs = [float(i) / s_times for i in times]
            # abhisha: get the final samples (for FPS, for downsampling) by applying the prob penalty and the fscore down sample
            samples = np.random.choice(a=cascade_nodes, size=round((no_samples)*f_score), p=probs)  # multiplied by fair score for fps
            # samples = np.random.choice(a=cascade_nodes, size=round((no_samples) * f_score), p=probs) # direct sampling for fac
        # ----- Store train set
        op_id = initiators[0]
        #print('@store_samples::write ', 'len(samples)=', len(samples))
        #print('@store_samples ', 'writing to ', train_set)
        for i in samples:
            train_set.write(str(op_id) + "," + i + "," + str(casc_len) + "," + str(f_score) + "\n")

    #print('@store_samples ', 'FINISHED writing to ', train_set)


In [29]:
# define main control parameters
input_fn = 'weibo'
#sampling_perc = 120
sampling_perc = 5
attribute = 'gender'

In [30]:
# load the network graph into memory
# expect this to take 15 minutes
print("Reading the network")
txt_file_path = '/opt/data/weibo_network.txt'
g = ig.Graph.Read_Ncol(txt_file_path)
print("Completed reading the network.")

Reading the network
Completed reading the network.


In [31]:
# check that network graph has been loaded correctly
print('number of edges in graph: ', g.ecount())

number of edges in graph:  225877808


In [32]:
# set training & attribute files; load attribute dictionary
train_set_file = f'{data_path}Data/Weibo/Init_Data/train_set_fair_gender_fps_v4.txt'  # set the train_set file according to different attribute
attribute_csv = f'{data_path}Data/Weibo/Init_Data/profile_gender.csv'      #  !!! use v6  set attribute csv file to write corresponding attribute
!ls -lah $attribute_csv
user_attribute_dict = get_attribute_dict(input_fn, attribute_csv, attribute)

-rw------- 1 root root 33M Jul 10  2022 /gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/profile_gender.csv


In [33]:
# group statistics
attribute_grouped = defaultdict(list)
for k, v in user_attribute_dict.items():
    attribute_grouped[v].append(k)
print('generate grouped nodes')

generate grouped nodes


In [36]:
# check the total number of training cascades
num_lines = 0
with open(f'{data_path}Data/Weibo/Init_Data/train_cascades.txt', "r") as f:
    for line in f:
      num_lines += 1
print('num cascades: ', num_lines)

num cascades:  97034


In [37]:
# sample a smaller number of cascades for testing
# yuting: sample some percentage which is fair, but also subselect the other files with the same nodes that show up in our sampling, for example user profile. The other thing she asked us to do was sample intelligently, meaning that we should sample users who have large-ish cascades so we can actually evaluate an influencer, since the graph may be naturally sparse
!head -50 '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/train_cascades.txt' > '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/train_cascades_small.txt'

In [47]:
!cat '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/train_cascades.txt' | awk -v OFS="," '{ print length, $0; }' | sort -t',' -g | tail -500 | cut -f 2- -d ',' | tail -25 > '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/train_cascades_small.txt'

In [48]:
# run feature extraction

with open("time_log.txt", "a") as log:
  with open(f'{data_path}Data/Weibo/Init_Data/train_cascades_small.txt', "r") as f, open(train_set_file, "w") as train_set:
  #with open("time_log.txt", "a") as log, open('/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/train_cascades.txt', "r") as f, open(train_set_file, "w") as train_set:
    # ----- Initialize features
    deleted_nodes = []
    g.vs["Cascades_started"] = 0
    g.vs["Cumsize_cascades_started"] = 0
    g.vs["Cascades_participated"] = 0
    log.write(f" net:{input_fn}\n")
    idx = 0

    start = time.time()
    # ---------------------- Iterate through cascades to create the train set
    for line in f:

        cascade = line.replace("\n", "").split(";")
        if input_fn == 'weibo':
            cascade_nodes = list(map(lambda x: x.split(" ")[0], cascade[1:]))
            # abhisha: why are we choosing this date 2011-10-28? Is this some cutoff date for the message cascades?
            # hypothesis is that it is the earliest timestamp of the cascades
            cascade_times = list(map(lambda x: int(((datetime.strptime(x.replace("\r", "").split(" ")[1],
                                                                        '%Y-%m-%d-%H:%M:%S') - datetime.strptime(
                "2011-10-28", "%Y-%m-%d")).total_seconds())), cascade[1:]))
        else:
            cascade_nodes = list(map(lambda x:  x.split(" ")[0],cascade))
            cascade_times = list(map(lambda x:  int(x.replace("\r","").split(" ")[1]),cascade))

        # ---- Remove retweets by the same person in one cascade
        cascade_nodes, cascade_times = remove_duplicates(cascade_nodes, cascade_times)

        # ---------- Dictionary nodes -> cascades
        op_id, op_time = cascade_nodes[0], cascade_times[0]

        try:
            g.vs.find(name=op_id)["Cascades_started"] += 1
            g.vs.find(op_id)["Cumsize_cascades_started"] += len(cascade_nodes)
        except:
            deleted_nodes.append(op_id)
            continue

        # abhisha: why are we skipping this and making it less than 3
        if len(cascade_nodes) < 3:
            continue
        initiators = [op_id]

        store_samples(input_fn, cascade_nodes[1:], cascade_times[1:], initiators, train_set, op_time,
                      user_attribute_dict, attribute_grouped, attribute, sampling_perc)

        idx += 1
        if idx % 1000 == 0:
            print("-------------------", idx)

    print(f"Number of nodes not found in the graph: {len(deleted_nodes)}")
  log.write(f"Feature extraction time:{str(time.time() - start)}\n")

  print("Evaluating fairness score of each influencer in train_cascades")

  kcores = g.shell_index()
  log.write(f"K-core time:{str(time.time() - start)}\n")
  a = np.array(g.vs["Cumsize_cascades_started"], dtype=np.float64)
  b = np.array(g.vs["Cascades_started"], dtype=np.float64)

  np.seterr(divide='ignore', invalid='ignore')

  # ------ Store node characteristics
  node_features = f'{data_path}Data/Weibo/FPS/node_features_small.csv'
  #node_feature_fair_age = '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/node_feature_age_fps.csv'
  # node_feature_fair_age = '/media/yuting/TOSHIBA EXT/digg/sampled/node_feature_age_fps.csv'
  pd.DataFrame({"Node": g.vs["name"],
                "Kcores": kcores,
                "Participated": g.vs["Cascades_participated"],
                "Avg_Cascade_Size": a / b}).to_csv(node_features, index=False)

  print("Finished storing node characteristics")

  # # ------ Derive incremental node dictionary
  # graph = pd.read_csv('/media/yuting/TOSHIBA EXT/digg/' + fn + "_network.txt", sep=" ")
  # graph.columns = ["node1", "node2", "weight"]
  # all = list(set(graph["node1"].unique()).union(set(graph["node2"].unique())))
  # dic = {int(all[i]): i for i in range(0, len(all))}
  # with open('/media/yuting/TOSHIBA EXT/digg/' + fn + "_incr_dic.json", "w") as json_file:
  #     json.dump(dic, json_file)



Number of nodes not found in the graph: 0
Evaluating fairness score of each influencer in train_cascades
Finished storing node characteristics


In [49]:
# count number of lines in the new nodes file, compare it to the original
!wc -l '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/node_features.csv'
!wc -l '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/node_features_small.csv'

1170689 /gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/node_features.csv
1170689 /gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/node_features_small.csv


In [50]:
# ensure you have train cascades
!ls -lah '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/train_cascades.txt'

-rw------- 1 root root 373M Jul 10  2022 /gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/train_cascades.txt


In [51]:
# check for nodes not found in the dictionaries
for node in cascade_nodes:
  if node in user_attribute_dict:
    print('found ', node)
  else:
    print('not found ', node)


Streaming output truncated to the last 5000 lines.
found  11247
found  1598741
found  1058298
found  441611
found  1314272
found  664088
found  356655
found  1083434
found  1593505
found  130096
found  42763
found  371634
found  1117471
found  1025856
found  1299603
found  1555970
found  430313
found  857678
found  186424
found  732759
found  104755
found  284092
found  1124868
found  1453462
found  1515991
found  342502
found  1109296
found  1522141
found  845137
found  1373913
found  296749
found  30983
found  230663
found  348442
found  365943
found  1015474
found  1142256
found  1458900
found  1143874
found  332438
found  818796
found  95077
found  619002
found  889114
found  482960
found  20959
found  941236
found  1130114
found  1003181
found  1064624
found  868987
found  96865
found  398925
found  606041
found  117832
found  353904
found  291329
found  1541638
found  959969
found  263659
found  853509
found  596346
found  1519771
found  363024
found  534201
found  959203
found  

In [52]:
# run the remainder of the script: preprocessing for IM

#prefix = '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/'
# change to correct directory
%cd '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/'
#print(input_fn)
with open("time_log.txt", "a") as log:
  # preprocessing.run(input_fn, log)
  # extract_feats_and_trainset.run(input_fn, 'region', sampling_perc, log)
  preprocess_for_imm.run(input_fn, log)
  # rank_nodes.run(input_fn)
  # infector.run(input_fn, learning_rate, n_epochs, embedding_size, num_neg_samples, log)
  # iminfector.run(input_fn, embedding_size, log)
  # evaluation.run2(input_fn, log, 'region')

/gdrive/MyDrive/FairInfluenceMaximization/data/Data


/gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models/preprocess_for_imm.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  graph = graph.drop(graph.columns[2], 1)


In [53]:
# switch context to local implementation
# should run autoreload to refresh context in memory, else a file change in .py in drive is not sufficient to load latest
%load_ext autoreload

import importlib
sys.path.append('/gdrive/MyDrive/FairInfluenceMaximization/code/')
#import fair_at_scale.models.infector as infector
import fair_at_scale.models.infector
import fair_at_scale.models.rank_nodes
import fair_at_scale.models.fair_iminfector
import fair_at_scale.models.evaluation
importlib.reload(fair_at_scale.models.infector)
importlib.reload(fair_at_scale.models.rank_nodes)
importlib.reload(fair_at_scale.models.evaluation)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'fair_at_scale.models.evaluation' from '/gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models/evaluation.py'>

In [54]:
# run the remainder of the script: node ranking

#prefix = '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/'
# change to correct directory
%cd '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/'
features_dir = '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/'
#print(input_fn)
with open("time_log.txt", "a") as log:
  # preprocessing.run(input_fn, log)
  # extract_feats_and_trainset.run(input_fn, 'region', sampling_perc, log)
  # preprocess_for_imm.run(input_fn, log)
  fair_at_scale.models.rank_nodes.run(input_fn, features_dir)
  # infector.run(input_fn, learning_rate, n_epochs, embedding_size, num_neg_samples, log)
  # iminfector.run(input_fn, embedding_size, log)
  # evaluation.run2(input_fn, log, 'region')

/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS


In [55]:
# run the remainder of the script: infector
# should take about 11 minutes

#prefix = '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/'
# change to correct directory
%cd '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/'
#print(input_fn)
with open("time_log.txt", "a") as log:
  # preprocessing.run(input_fn, log)
  # extract_feats_and_trainset.run(input_fn, 'region', sampling_perc, log)
  # preprocess_for_imm.run(input_fn, log)
  # rank_nodes.run(input_fn)
  fair_at_scale.models.infector.run(input_fn, learning_rate, n_epochs, embedding_size, num_neg_samples, log)
  # iminfector.run(input_fn, embedding_size, log)
  # evaluation.run2(input_fn, log, 'region')

/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS
6
1170688
Time taken for the weibo infector:2704.351673603058



In [56]:
# run the remainder of the script: IMinfector
# should take about 11 minutes

#prefix = '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/'
# change to correct directory
%cd '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/'
#print(input_fn)
with open("time_log.txt", "a") as log:
  # preprocessing.run(input_fn, log)
  # extract_feats_and_trainset.run(input_fn, 'region', sampling_perc, log)
  # preprocess_for_imm.run(input_fn, log)
  # rank_nodes.run(input_fn)
  # fair_at_scale.models.infector.run(input_fn, learning_rate, n_epochs, embedding_size, num_neg_samples, log)
  fair_at_scale.models.fair_iminfector.run(input_fn, embedding_size, log)
  # evaluation.run2(input_fn, log, 'region')

/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS
6
Time taken for the weibo IMInfector: 32.30178380012512



In [57]:
# run the remainder of the script: Evaluation
# should take about 11 minutes

#prefix = '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/'
# change to correct directory
%cd '/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/'
#print(input_fn)
with open("time_log.txt", "a") as log:
  # preprocessing.run(input_fn, log)
  # extract_feats_and_trainset.run(input_fn, 'region', sampling_perc, log)
  # preprocess_for_imm.run(input_fn, log)
  # rank_nodes.run(input_fn)
  # fair_at_scale.models.infector.run(input_fn, learning_rate, n_epochs, embedding_size, num_neg_samples, log)
  #fair_at_scale.models.fair_iminfector.run(input_fn, embedding_size, log)
  #evaluation.run(input_fn, log, 'region')
  fair_at_scale.models.evaluation.run(input_fn, log)

/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS
/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/Seeds/final_seeds.txt
------------------
Seeds found: 48
Seeds found: 89
Seeds found: 132
Seeds found: 173
Seeds found: 210
Seeds found: 233
Seeds found: 255
Seeds found: 281
Seeds found: 306
Seeds found: 334
Seeds found: 363
Seeds found: 393
Seeds found: 414
Seeds found: 441
Seeds found: 470
Seeds found: 494
Seeds found: 514
Seeds found: 544
Seeds found: 570
Seeds found: 592
Seeds found: 594
/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/FPS/Seeds/kcores_seeds.txt
------------------
Seeds found: 27
Seeds found: 49
Seeds found: 70
Seeds found: 98
Seeds found: 125
Seeds found: 151
Seeds found: 178
Seeds found: 201
Seeds found: 216
Seeds found: 239
Seeds found: 253
Seeds found: 278
Seeds found: 298
Seeds found: 325
Seeds found: 348
Seeds found: 373
Seeds found: 394
Seeds found: 406
Seeds found: 421
Seeds found: 446
Seeds found: 446
/gdrive/MyDrive/FairInf

In [ ]:
### MISCELLANEOUS WORK BELOW

In [ ]:
!grep 'train_set_fair_gender_fps_v2_new.txt' /gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models/*.py

/gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models/fair_iminfector.py:        # with open("/gdrive/MyDrive/FairInfluenceMaximization/data/weibodata/processed4maxmization/weibo/train_set_fair_gender_fps_v2_new.txt", "w") as ftp:
/gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models/infector.py:        with open('/gdrive/MyDrive/FairInfluenceMaximization/data/weibodata/processed4maxmization/weibo/train_set_fair_gender_fps_v2_new.txt', "r") as f:
/gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models/infector.py:                with open('/gdrive/MyDrive/FairInfluenceMaximization/data/weibodata/processed4maxmization/weibo/train_set_fair_gender_fps_v2_new.txt',"r") as f:


In [ ]:
# run the model
%cd /gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models/

%set_env PYTHONPATH=/env/python:/gdrive/MyDrive/FairInfluenceMaximization/code/IMINFECTOR/

!python main.py

In [ ]:
!grep 'count_distinct_nodes_influenced' /gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models/*.py

/gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models/evaluation.py:def count_distinct_nodes_influenced(seed_set_cascades: Dict) -> int:
/gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models/evaluation.py:                spreading_of_set[seed_set_size] = count_distinct_nodes_influenced(seed_set_cascades)
/gdrive/MyDrive/FairInfluenceMaximization/code/fair_at_scale/models/evaluation.py:                # spreading_of_set[seed_set_size] = count_distinct_nodes_influenced(seed_set_cascades)


In [ ]:
!find /gdrive/MyDrive/FairInfluenceMaximization/data/ -type f -name '*test_cascades*'

/gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/Init_Data/test_cascades.txt
/gdrive/MyDrive/FairInfluenceMaximization/data/__MACOSX/Data/Weibo/Init_Data/._test_cascades.txt


In [ ]:
!find /gdrive/MyDrive/FairInfluenceMaximization/data/ -type f -name 'profile_regionv3.csv'

In [ ]:
!head /gdrive/MyDrive/FairInfluenceMaximization/data/Data/Weibo/weibo_incr_dic.json

{"0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "11": 9, "12": 10, "13": 11, "15": 12, "16": 13, "17": 14, "18": 15, "19": 16, "20": 17, "21": 18, "22": 19, "24": 20, "26": 21, "27": 22, "30": 23, "31": 24, "33": 25, "34": 26, "36": 27, "37": 28, "40": 29, "42": 30, "43": 31, "44": 32, "45": 33, "46": 34, "48": 35, "49": 36, "50": 37, "51": 38, "52": 39, "53": 40, "54": 41, "57": 42, "58": 43, "59": 44, "60": 45, "61": 46, "62": 47, "63": 48, "64": 49, "65": 50, "67": 51, "69": 52, "70": 53, "71": 54, "72": 55, "74": 56, "75": 57, "77": 58, "90": 59, "91": 60, "92": 61, "93": 62, "94": 63, "96": 64, "99": 65, "101": 66, "103": 67, "105": 68, "106": 69, "107": 70, "112": 71, "113": 72, "116": 73, "117": 74, "118": 75, "119": 76, "121": 77, "124": 78, "125": 79, "126": 80, "127": 81, "128": 82, "129": 83, "130": 84, "132": 85, "133": 86, "134": 87, "135": 88, "136": 89, "137": 90, "138": 91, "140": 92, "141": 93, "142": 94, "143": 95, "144": 96, "145": 97, "147":

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

